# Outil de débogage interactif

**Description**

Ce notebook offre un outil interactif (widget) pour visualiser les évènements des matchs répertoriés par saison.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import json

In [ ]:
@interact
def interactive_debugger(season=['2016_2017', '2017_2018', '2018_2019', '2019_2020', '2020_2021'], season_type=['regular', 'playoff'], game_id=[], event=[]):
    """
    season: str
        Season to visualize. 
    season_type: str
        Season type to visualize. Can be `reguliere` or `eliminatoire`.
    game_id: str or int ?
        ID of the game to visualize.
    event: str or int ?
        Event number to visualize within the specified game.
    """
    # TODO
    info = json.load(open(f'../data/data_{season}_play_by_play.json'))
    